# 0. Load Packages, Import Data and First Print

# to-dos
Test the models 

1. Logistic Regression
2. Support Vector Machine (SVM)
3. K-Nearest Neighbors (KNN)
4. Decision Trees
5. Bagging Algorithms
6. Boosting Algorithms
7. Random Forests
8. Voting Classifier and Stacking
9. XGBoost

with 

-benchmark 
-choose one model
- F1 score seems to be good
- one model plant 
- take one the healthy 

finetuning with

- Gridsearch
- random select
- lazypredict 

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    BaggingClassifier, AdaBoostClassifier, RandomForestClassifier)
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from datetime import date
import os
import joblib


In [3]:

# Load the saved arrays
features_train = np.load("/Users/felix/Documents/Data Science/06_Offical_project_DS/may25_bds_plants/05_data/computed_data/2025_06_25_features_train.npy")
labels_train = np.load("/Users/felix/Documents/Data Science/06_Offical_project_DS/may25_bds_plants/05_data/computed_data/2025_06_25_labels_train.npy")


# Load the saved arrays
features_valid = np.load("/Users/felix/Documents/Data Science/06_Offical_project_DS/may25_bds_plants/05_data/computed_data/2025_06_25_features_valid.npy")
labels_valid = np.load("/Users/felix/Documents/Data Science/06_Offical_project_DS/may25_bds_plants/05_data/computed_data/2025_06_25_labels_valid.npy")


# Inspect the data
print("Train Features shape:", features_train.shape )
print("Train Labels shape:", labels_train.shape,"\n")

print("Test Features shape:", features_valid.shape)
print("Test Labels shape:", labels_valid.shape,"\n")

# Print a sample of the data
print("Train Sample feature vector:", features_train[0])
print("Train Sample label:", labels_train[0],"\n")

print("Test Sample feature vector:", features_valid[0])
print("Test Sample label:", labels_valid[0])

Train Features shape: (70295, 1280)
Train Labels shape: (70295,) 

Test Features shape: (17572, 1280)
Test Labels shape: (17572,) 

Train Sample feature vector: [0.         0.86447287 0.23696803 ... 0.25945228 0.80036694 0.5455655 ]
Train Sample label: Apple___Apple_scab 

Test Sample feature vector: [0.         1.9377055  0.         ... 0.         0.33774427 1.586906  ]
Test Sample label: Apple___Apple_scab


# 1. Filter the Data for Heallty Plants

### Training Data

In [4]:
# Filter condition: Boolean mask
mask = np.char.endswith(labels_train.astype(str), "healthy")

# Apply mask to features and labels
features_healthy_train = features_train[mask]
labels_healthy_train = labels_train[mask]

# Print a sample of the data
print("Sample feature vector:", features_healthy_train [0])
print("Sample label:", labels_healthy_train [0])
print(np.unique(labels_healthy_train))



Sample feature vector: [0.02920142 2.5676873  0.         ... 0.02060599 0.21142866 0.0756085 ]
Sample label: Apple___healthy
['Apple___healthy' 'Blueberry___healthy'
 'Cherry_(including_sour)___healthy' 'Corn_(maize)___healthy'
 'Grape___healthy' 'Peach___healthy' 'Pepper,_bell___healthy'
 'Potato___healthy' 'Raspberry___healthy' 'Soybean___healthy'
 'Strawberry___healthy' 'Tomato___healthy']


### Test Data

In [5]:
# Create Boolean mask for labels ending with 'healthy'
mask = np.char.endswith(labels_valid.astype(str), "healthy")

# Apply mask to filter features and labels
features_healthy_valid = features_valid[mask]
labels_healthy_valid = labels_valid[mask]

# Print a sample
print("Sample feature vector:", features_healthy_valid[0])
print("Sample label:", labels_healthy_valid[0])
print("All unique healthy labels:", np.unique(labels_healthy_valid))


Sample feature vector: [0.17777829 2.1144028  0.         ... 1.1448742  0.7059598  1.9669307 ]
Sample label: Apple___healthy
All unique healthy labels: ['Apple___healthy' 'Blueberry___healthy'
 'Cherry_(including_sour)___healthy' 'Corn_(maize)___healthy'
 'Grape___healthy' 'Peach___healthy' 'Pepper,_bell___healthy'
 'Potato___healthy' 'Raspberry___healthy' 'Soybean___healthy'
 'Strawberry___healthy' 'Tomato___healthy']


# 2. Training Basic Models

In [ ]:
# Encode labels
import random


le = LabelEncoder()
y_train = le.fit_transform(labels_healthy_train)
y_valid = le.transform(labels_healthy_valid)


Encoded labels (train): []


In [7]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Bagging": BaggingClassifier(),
    "Boosting": AdaBoostClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

# Train and evaluate
for name, model in models.items():
    model.fit(features_healthy_train, y_train)
    y_pred = model.predict(features_healthy_valid)
    acc = accuracy_score(y_valid, y_pred)
    print(f"\n{name} - Accuracy: {acc:.4f}")
    print(classification_report(y_valid, y_pred, target_names=le.classes_))





Logistic Regression - Accuracy: 0.9937
                                   precision    recall  f1-score   support

                  Apple___healthy       1.00      0.99      0.99       502
              Blueberry___healthy       1.00      0.99      0.99       454
Cherry_(including_sour)___healthy       0.99      1.00      0.99       456
           Corn_(maize)___healthy       1.00      1.00      1.00       465
                  Grape___healthy       1.00      1.00      1.00       423
                  Peach___healthy       0.99      1.00      1.00       432
           Pepper,_bell___healthy       0.99      0.99      0.99       497
                 Potato___healthy       0.99      0.99      0.99       456
              Raspberry___healthy       0.99      0.99      0.99       445
                Soybean___healthy       0.99      0.99      0.99       505
             Strawberry___healthy       1.00      1.00      1.00       456
                 Tomato___healthy       1.00      0.99     

/Users/felix/Documents/Data Science/06_Offical_project_DS/may25_bds_plants/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:15:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



XGBoost - Accuracy: 0.9840
                                   precision    recall  f1-score   support

                  Apple___healthy       0.99      0.98      0.99       502
              Blueberry___healthy       0.98      0.98      0.98       454
Cherry_(including_sour)___healthy       0.98      0.99      0.99       456
           Corn_(maize)___healthy       1.00      1.00      1.00       465
                  Grape___healthy       1.00      1.00      1.00       423
                  Peach___healthy       0.97      0.99      0.98       432
           Pepper,_bell___healthy       0.99      0.96      0.97       497
                 Potato___healthy       0.98      0.97      0.97       456
              Raspberry___healthy       0.98      0.99      0.99       445
                Soybean___healthy       0.97      0.98      0.97       505
             Strawberry___healthy       0.99      0.98      0.98       456
                 Tomato___healthy       0.99      0.98      0.98       

### 3. Save the Models

In [16]:
# Directory to save models
save_dir = "/Users/felix/Documents/Data Science/06_Offical_project_DS/may25_bds_plants/02_models/01_first_models"
today = date.today().isoformat()

# Save models with sanitized filenames
for name, model in models.items():
    safe_name = name.replace(" ", "_")  # Replace spaces with underscores
    filename = f"{save_dir}/{today}_{safe_name}.joblib"
    joblib.dump(model, filename)

print("Models saved successfully.")


Models saved successfully.
